# Self-driving cab as a Reinforcement Learning problem
### using Q-Learning

<p>
Many of us can't have self driving car as of now. Don't worry, we can have baby step now.
</p>
```As History is the witness, Software come long before Hardware``` ;p

<a href="https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/">Source</a>

# My Reaction

### initial thought
> After training for half an hour CNN model to recognise 28x28 matrix 60,000 of them getting 98% accuracy and then reading the problem of pickup and dropping. I thought, how on earth a small AI model will learn without  large error. Seriously I didn't expect much.

### after reaction
> <p>seeing the score didn't give any feelings. But when I run(visuals) the random one and the trained one, It felt like, it is real, someone is controlling it, Yeah I know there wasn't.
It was so accurate.</p> It really felt like taking a baby step in understanding human mind.

### long after thought
<p> After reading the math and getting the working knowledge, I understand it just the magic of almighty math. It just learns all the possibilities to solve a state. Sort of Overfitting in the lang. of Deep Learning. 
</p>
*But Hey, it learned right!*
<p>
When you DIY, then only you come to know how powerful this thingy can go.
</p>

```Its like typing magic spell to create artificial human```

In [1]:
from IPython.display import clear_output
from time import sleep
import numpy as np
from tqdm import tqdm

import gym

In [2]:
env = gym.make('Taxi-v2').env

In [3]:
env.reset()
env.render()

print("Action Space {}".format(env.action_space))
print("Action Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
Action Space Discrete(500)


- **Yellow:** The taxi is empty.
- **Green:** The taxi is having passenger
- **R,G,B,Y:** Drop or pick-up location (Not Colors)
- **| (pipe):** A wall which car cannot cross

In [4]:
env.P[env.s]

{0: [(1.0, 241, -1, False)],
 1: [(1.0, 41, -1, False)],
 2: [(1.0, 161, -1, False)],
 3: [(1.0, 121, -1, False)],
 4: [(1.0, 141, -10, False)],
 5: [(1.0, 141, -10, False)]}

It is the ```step x [probability, next state, reward, Done?]``` matrix of where the taxi is.

So, 6 possibilities at a point and its consequences.

In [5]:
def train_Q_driver(environment, epochs, alpha=0.1, gamma=0.6, epsilon=0.1):
    q_table = np.zeros([
        environment.observation_space.n,
        environment.action_space.n
    ])
    
    all_epochs = []
    all_penalties = []
    
    for i in tqdm(range(1, epochs)):
        state = env.reset()
        
        epochs, penalties, reward = 0, 0, 0
        done = False
        
        while not done:
            if np.random.uniform(0, 1) < epsilon:
                action = environment.action_space.sample()
            else:
                action = np.argmax(q_table[state])
                
            next_state, reward, done, info = env.step(action)
            
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1
    return q_table
    

### we are using this tiny formula to make our driver learn
$$Q({\small state}, {\small action}) \leftarrow (1 - \alpha) Q({\small state}, {\small action}) +\alpha\Big({\small reward} + \gamma \max_{a} Q({\small next \ state}, {\small all \ actions})\Big)$$

In [6]:
def play_one_episode(agent=None, q_table=None, Output=False):
    
    state = env.reset()
    epochs = 0
    penalties, reward = 0, 0
    frames = []
    done = False
    while not done:
        if(agent):
            action = np.argmax(q_table[state])
        else:
            action = env.action_space.sample()
        state, reward, done, _ = env.step(action)

        if (reward == -10):
            penalties += 1

        if Output:
            frames.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
            })

        epochs += 1

    return frames, epochs, penalties

In [7]:
def print_frames(frames, interval=0.1):
    epochs = len(frames)
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}/{epochs}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(interval)
        

In [8]:
def evaluate_Q_driver(q_table):
    total_epochs, total_penalties = 0, 0
    episodes = 3
    
    for _ in tqdm(range(episodes)):
        _, e, p = play_one_episode(1, q_table)
        
        total_penalties += p
        total_epochs += e
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_epochs / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")

In [9]:
q_table = train_Q_driver(env, 10001)

100%|██████████| 10000/10000 [00:04<00:00, 2368.70it/s]


# Just have a look once
### See once how random agent drives taxi Vs our trained one

> Here goes the Random

In [10]:
f, e, p = play_one_episode(Output=True)
print_frames(f)
print("epochs: {}, penalties: {}".format(e, p))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 2534/2534
State: 479
Action: 5
Reward: 20
epochs: 2534, penalties: 823


> Here goes the Almighty Math

In [11]:
f, e, p = play_one_episode(1, q_table, Output=True)
print_frames(f, 0.5)
print("epochs: {}, penalties: {}".format(e, p))


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 13/13
State: 16
Action: 5
Reward: 20
epochs: 13, penalties: 0


In [12]:
evaluate_Q_driver(q_table)

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:

"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
    

KeyboardInterrupt: 

In [16]:
env.step(0)

(409, -1, False, {'prob': 1.0})